In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from signals import *
aapl = yf.download('AAPL', start='2010-01-01', end='2018-12-31')
adjusted_close_prices = aapl['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [2]:
BB_window = 20
rolling_mean = adjusted_close_prices.rolling(window=BB_window).mean()
rolling_std = adjusted_close_prices.rolling(window=BB_window).std()
upper_band = rolling_mean + rolling_std
lower_band = rolling_mean - rolling_std
df = pd.DataFrame(np.array([adjusted_close_prices, upper_band, lower_band]).T, columns=['price', 'upper', 'lower'], index=adjusted_close_prices.index)
df['next_5_ret'] = (df['price'] / df['price'].shift(5) - 1).shift(-5)
df.reset_index(inplace=True)
df.Date = df.Date.dt.strftime('%Y%m%d')

In [5]:
df = simple_signal(df)

In [6]:
df['label_1'] = np.nan
df['label_2'] = np.nan
for i in range(BB_window, len(df)):
    if df.loc[i, 'price'] < df.loc[i, 'lower']:
        df.loc[i, 'label_1'] = 'buy'
    if df.loc[i, 'price'] > df.loc[i, 'upper']:
        df.loc[i, 'label_1'] = 'sell'

for i in range(len(df)-5):
    if df.loc[i, 'next_5_ret'] < -0.02:
        df.loc[i, 'label_2'] = 'sell'
    if df.loc[i, 'next_5_ret'] > 0.02:
        df.loc[i, 'label_2'] = 'buy'

df

C:\Users\hefzh\AppData\Local\Temp\ipykernel_17124\2666365270.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'buy' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'label_1'] = 'buy'
C:\Users\hefzh\AppData\Local\Temp\ipykernel_17124\2666365270.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'sell' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'label_2'] = 'sell'


,Date,price,upper,lower,next_5_ret,5_MA,5_EMA,10_MA,10_EMA,20_MA,...,dema,HMA,WMA,PPO,DPO,RSI,ROC,PROC,label_1,label_2
0,20100104,6.461977,NaN,NaN,-0.018224,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20100105,6.473147,NaN,NaN,-0.031066,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sell
2,20100106,6.370185,NaN,NaN,-0.001517,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20100107,6.358410,NaN,NaN,-0.005461,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20100108,6.400681,NaN,NaN,-0.028540,6.412880,6.406035,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,20181221,36.073055,42.764921,38.776021,NaN,38.217862,37.905114,39.301989,39.064109,40.770471,...,37.704116,38.478563,39.888717,-5.557226,1.043925,24.833742,-14.692412,-14.692412,buy,NaN
2259,20181224,35.139694,42.819023,38.112604,NaN,37.398902,36.983308,38.757053,38.350579,40.465814,...,36.970409,37.674705,39.352453,-6.038062,-0.142392,22.983617,-15.962688,-15.962688,buy,NaN
2260,20181226,37.614288,42.670986,37.843024,NaN,36.972906,37.193635,38.482789,38.216708,40.257005,...,36.802101,37.481705,39.080879,-5.854340,0.332053,36.492586,-6.718510,-6.718510,buy,NaN
2261,20181227,37.370174,42.511788,37.569286,NaN,36.746027,37.252481,38.172866,38.062793,40.040537,...,36.627409,37.303329,38.805942,-5.697290,0.316385,35.825013,-7.930422,-7.930422,buy,NaN


In [ ]:
fig_window_size_list = [20]
fig_width = 0.6
fig_height = 0.64
dpi = 100

for window_size in fig_window_size_list:
    dir_path = f'window_{window_size}_plots'
    os.makedirs(dir_path, exist_ok=True)
    for i in range(window_size, df.shape[0]):
        label1 = df['label_1'].iloc[i]
        label2 = df['label_2'].iloc[i]
        next_5_ret = df['next_5_ret'].iloc[i]
        df_window = df.iloc[i-window_size: i].copy()
        start = df_window.Date.iloc[0]
        end = df_window.Date.iloc[-1]
        plot_path = os.path.join(dir_path, f"window{window_size}_{start}_{end}_{label1}_{label2}_{next_5_ret:.6f}.png")
        plt.figure(figsize=(fig_width, fig_height), dpi=dpi)
        plt.plot(df_window['price'], color='blue')
        plt.plot(df_window['upper'], color='red')
        plt.plot(df_window['lower'], color='green')
        plt.xticks([])
        plt.yticks([])
        plt.savefig(plot_path)
        plt.close()